In [ ]:
import json
import numpy as np
import pandas as pd
from cgmres import cartpole_error_monitoring, common
from autogenu import RK4, Logger, Plotter

def load_config(config_name):
    with open(f'./config/{config_name}', 'r') as f:
        return json.load(f)

def load_data(project_name):
    filenames = [f'./log/{project_name}_{type_}.log' for type_ in ['uopt', 'x', 't']]
    df_list = [pd.read_csv(filename, delim_whitespace=True, header=None) for filename in filenames]
    return [df.values for df in df_list]  # u_matrix, x_matrix, t

def initialize_solver(config, ocp):
    settings = common.SolverSettings()
    settings.sampling_time = config["sampling_time"]
    settings.zeta = config["zeta"]
    settings.finite_difference_epsilon = config["finite_difference_epsilon"]
    settings.max_iter = config["max_iterations"]
    settings.opterr_tol = config["tolerance"]
    settings.verbose_level = 1

    horizon = common.Horizon(config["Tf"])  # fixed length
    mpc = cartpole_error_monitoring.SingleShootingCGMRESSolver(ocp, horizon, settings)

    initializer = cartpole_error_monitoring.ZeroHorizonOCPSolver(ocp, settings)
    initializer.set_uc(np.array([config["solution_initial_guess"]]))
    initializer.solve(config["initial_time"], np.zeros(ocp.nx))

    mpc.set_uc(initializer.ucopt)

    return mpc

def initialize_logger(log_dir='./log', log_name='test_numelic_F'):
    global logger
    logger = Logger(log_dir=log_dir, log_name=log_name)

def run_simulation(config, mpc, ocp):
    u_opt_matrix, x_matrix, t_series = load_data(project_name)
    simulation_time = config["simulation_length"]
    sampling_time = config["sampling_time"]
    t = config["initial_time"]
    # x = np.zeros(ocp.nx)
    x = np.array(x_matrix[0])

    

    for i in range(int(simulation_time/sampling_time)-1):
        # u = mpc.uopt[0]
        u = np.array([u_opt_matrix[i][0]])
        print(i)
        # x1 = RK4(ocp, t, sampling_time, x, u)
        x1 = np.array(x_matrix[i+1])
        mpc.update(t, x)
        logger.save(t, x, u, mpc.opt_error())
        x = x1
        t = t + sampling_time
        print(f't: {t}, x: {x}')

if __name__ == "__main__":
    project_name = "cartpole_error_monitoring"
    config_name = 'config_20230722.json'

    ocp = cartpole_error_monitoring.OCP()
    config = load_config(config_name)
    mpc = initialize_solver(config, ocp)
    initialize_logger()

    run_simulation(config, mpc, ocp)
    print("\n======================= MPC used in this simulation: =======================")
    # print(mpc)